# Final Project: Do baggage fees delay flights?

## A. Import Data

###1. Import 'On-Time Performance' flight data from BTS (Bureau of Transportation Statistics) for a 12-month-period. 
####Data used was for June 2014 - May 2015
####Source: http://www.transtats.bts.gov/Fields.asp?Table_ID=236


In [1]:
import pandas as pd
import time
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from scipy import stats

####Import the monthly data sets and combine them into one (data_raw).

In [2]:
data_2014_6 = pd.read_csv("data/On_Time_On_Time_Performance_2014_6.csv")
data_2014_7 = pd.read_csv("data/On_Time_On_Time_Performance_2014_7.csv")
data_2014_8 = pd.read_csv("data/On_Time_On_Time_Performance_2014_8.csv")
data_2014_9 = pd.read_csv("data/On_Time_On_Time_Performance_2014_9.csv")
data_2014_10 = pd.read_csv("data/On_Time_On_Time_Performance_2014_10.csv")
data_2014_11 = pd.read_csv("data/On_Time_On_Time_Performance_2014_11.csv")
data_2014_12 = pd.read_csv("data/On_Time_On_Time_Performance_2014_12.csv")
data_2015_1 = pd.read_csv("data/On_Time_On_Time_Performance_2015_1.csv")
data_2015_2 = pd.read_csv("data/On_Time_On_Time_Performance_2015_2.csv")
data_2015_3 = pd.read_csv("data/On_Time_On_Time_Performance_2015_3.csv")
data_2015_4 = pd.read_csv("data/On_Time_On_Time_Performance_2015_4.csv")
data_2015_5 = pd.read_csv("data/On_Time_On_Time_Performance_2015_5.csv")


//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (48,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning:

Columns (77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
#Combine the datasets:
data_raw= ()
data_raw = data_2014_6.append([data_2014_7, data_2014_8, data_2014_9, data_2014_10,data_2014_11, data_2014_12, 
                                data_2015_1, data_2015_2, data_2015_3, data_2015_4, data_2015_5], ignore_index=True)



##Add additional variables to dataset

###2. Add Airline Descriptions
####The BTS "On-Performance" data sets do not contain the full name of the airlines and therefore an additional variable ('Description') was added through a lookup table (also obtained from BTS).


In [4]:
Airline_ID = pd.read_csv("data/L_AIRLINE_ID.csv", skipinitialspace=True)
Airline_ID.head()
data_full= pd.merge(data_raw, Airline_ID, how='left', left_on='AirlineID', right_on='Code')


###3. Add coordinates (latitude and longitude) for each airport in the data set. 
####This was in order to calculate the distance between airports and account for the jet stream when calculating target times for each route
####Source: http://datahub.io/dataset/global-airports 

In [5]:
#Get data on each airport. Read the csv file in iterations, and filter by US airports only to reduce loading time.
iter_csv= pd.read_csv("data/global_airports.csv", iterator=True, chunksize=1000)
airport_data = pd.concat([chunk[chunk['country'] == 'United States'] for chunk in iter_csv])
airport_data.head()


,airport_id,name,city,country,iata_faa,iaco,latitude,longitude,altitude,zone,dst
318,6891,Putnam County Airport,Greencastle,United States,4I7,NaN,39.633556,-86.813806,842,-5,U
104,6890,Dowagiac Municipal Airport,Dowagiac,United States,C91,NaN,41.992934,-86.128012,748,-5,U
121,6889,Cambridge Municipal Airport,Cambridge,United States,CDI,NaN,39.975028,-81.577583,799,-5,U
470,6885,Door County Cherryland Airport,Sturgeon Bay,United States,SUE,NaN,44.843667,-87.421556,725,-6,U
507,6884,Shoestring Aviation Airfield,Stewartstown,United States,0P2,NaN,39.794824,-76.647191,1000,-5,U


In [6]:
#Add latitude and longitude of each airport to the data set covering all routes. 
airport_location = airport_data[["iata_faa","latitude","longitude"]]
airport_location.head()
data1=()

##Add coordinates of Origin airport
data1 = pd.merge(data_full, airport_location, how='left', left_on="Origin", right_on="iata_faa",left_index=False, right_index=False, sort=True)
data1.rename(columns = {'latitude': 'lat_origin', 'longitude': 'lon_origin'}, inplace = True)

#Add coordinates of Destination airport
data2 = pd.merge(data1, airport_location, how='left', left_on="Dest", right_on="iata_faa",left_index=False, right_index=False, sort=True)
data2.rename(columns = {'latitude': 'lat_dest', 'longitude': 'lon_dest'}, inplace = True)
data2.drop(['iata_faa_x','iata_faa_y'], axis=1, inplace=True)
data_full=data2
data_full.head() 

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div5WheelsOff,Div5TailNum,Unnamed: 109,Code,Description,Des_abbrev,lat_origin,lon_origin,lat_dest,lon_dest
0,2014,2,6,1,7,2014-06-01,DL,19790,DL,N981AT,...,NaN,NaN,NaN,19790,Delta Air Lines Inc.,DL,33.636719,-84.428067,40.652083,-75.440806
1,2014,2,6,1,7,2014-06-01,DL,19790,DL,N955AT,...,NaN,NaN,NaN,19790,Delta Air Lines Inc.,DL,33.636719,-84.428067,40.652083,-75.440806
2,2014,2,6,2,1,2014-06-02,DL,19790,DL,N995AT,...,NaN,NaN,NaN,19790,Delta Air Lines Inc.,DL,33.636719,-84.428067,40.652083,-75.440806
3,2014,2,6,3,2,2014-06-03,DL,19790,DL,N924AT,...,NaN,NaN,NaN,19790,Delta Air Lines Inc.,DL,33.636719,-84.428067,40.652083,-75.440806
4,2014,2,6,3,2,2014-06-03,DL,19790,DL,N966AT,...,NaN,NaN,NaN,19790,Delta Air Lines Inc.,DL,33.636719,-84.428067,40.652083,-75.440806


In [7]:
#Check to make sure there was data for each airport of origin and destination. 
data_full[data_full.lon_origin == np.nan ]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div5WheelsOff,Div5TailNum,Unnamed: 109,Code,Description,Des_abbrev,lat_origin,lon_origin,lat_dest,lon_dest


## B. Data Exploration & Processing

###1. Find size of data set and variable names

In [8]:
len(data_full)

5816340

In [9]:
list(data_full)

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'UniqueCarrier',
 'AirlineID',
 'Carrier',
 'TailNum',
 'FlightNum',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'FirstDepTime',
 'TotalAddGTime',
 

In [10]:
#Look at the different departure and arrival time variables to see how they differ.
data_full[['CRSDepTime','DepTime', 'DepTimeBlk' ,'DepDelay','DepDelayMinutes','CRSArrTime','ArrTime', 'CRSElapsedTime','ActualElapsedTime']].head()

,CRSDepTime,DepTime,DepTimeBlk,DepDelay,DepDelayMinutes,CRSArrTime,ArrTime,CRSElapsedTime,ActualElapsedTime
0,1339,1335,1300-1359,-4,0,1537,1533,118,118
1,2045,2042,2000-2059,-3,0,2239,2238,114,116
2,2045,2057,2000-2059,12,12,2239,2250,114,113
3,1339,1338,1300-1359,-1,0,1537,1538,118,120
4,2045,2037,2000-2059,-8,0,2239,2244,114,127


###2. Find airlines that fly the 20 most common routes
####i. Find the 20 most common routes.

In [11]:
data_route=data_full[["Origin", "Dest"]]
route_grouped = data_route.groupby(["Origin", "Dest"]).size()
DF_data_route = pd.DataFrame(route_grouped)
DF_data_route.rename(columns = {0: 'Count'}, inplace = True)
data_route_sorted= DF_data_route.sort('Count', axis=0 , ascending=False)
twenty_routes = data_route_sorted.reset_index().head(20)
twenty_routes

,Origin,Dest,Count
0,SFO,LAX,13386
1,LAX,JFK,13374
2,JFK,LAX,13373
3,LAX,SFO,13288
4,LAS,LAX,11252
5,LAX,LAS,10910
6,LGA,ORD,10124
7,ORD,LGA,10096
8,HNL,OGG,9560
9,OGG,HNL,9500


####ii. Get a list of the carriers that fly those 20 routes.

In [12]:
twenty_only=twenty_routes.drop('Count', axis=1)
data_toproutes= pd.merge(data_full, twenty_only, how='inner', on=['Origin','Dest'])
data_toproutes[['Origin','Dest','UniqueCarrier','AirlineID','Carrier']].head()


,Origin,Dest,UniqueCarrier,AirlineID,Carrier
0,LGA,ATL,DL,19790,DL
1,LGA,ATL,DL,19790,DL
2,LGA,ATL,DL,19790,DL
3,LGA,ATL,DL,19790,DL
4,LGA,ATL,DL,19790,DL


In [13]:
print data_toproutes.UniqueCarrier.unique()
print data_toproutes.AirlineID.unique()

['DL' 'WN' 'FL' 'EV' 'MQ' 'F9' 'NK' 'B6' 'US' 'HA' 'AA' 'VX' 'UA' 'OO']
[19790 19393 20437 20366 20398 20436 20416 20409 20355 19690 19805 21171
 19977 20304]


####iii. Get the full names for each of these airlines

In [14]:
print data_toproutes.Description.unique()

['Delta Air Lines Inc.' 'Southwest Airlines Co.'
 'AirTran Airways Corporation' 'ExpressJet Airlines Inc.' 'Envoy Air'
 'Frontier Airlines Inc.' 'Spirit Air Lines' 'JetBlue Airways'
 'US Airways Inc.' 'Hawaiian Airlines Inc.' 'American Airlines Inc.'
 'Virgin America' 'United Air Lines Inc.' 'SkyWest Airlines Inc.']


### 3. Map flights operated by regional carriers
####The 3 major regional carriers are ExpressJet Airlines Inc., Envoy Air, and SkyWest Airlines Inc. I adopt FiveThiryEight's treatment and mapping of these:

    Envoy Air assigned to American Airlines
    
    "ExpressJet flight numbers are assigned as follows:
        Flight numbers 2500-2574 to American
        Flight numbers 3255-4868 to United
        Flight numbers 4869-5632 to Delta
        Flight numbers 5660-6189 to United
        And SkyWest flight numbers are assigned as follows:

        Flight numbers 2575-2649 to American
        Flight numbers 2901-2974 to US Airways
        Flight numbers 3448-3499 to Alaska
        Flight numbers 4438-4859 to Delta
        Flight numbers 4965-6539 to United
        Flight numbers 6550-6629 to US Airways
        Flight numbers 7362-7439 to Delta
        Flight numbers outside these ranges may represent irregular operations and are discarded. "
Source: http://fivethirtyeight.com/features/fastest-airlines-fastest-airports/ 

In [15]:
#Look at the flights that would be affected by this mapping change.
subset = data_toproutes[data_toproutes['Description'].isin(['ExpressJet Airlines Inc.', 'SkyWest Airlines Inc.','Envoy Air'])]
subset[['FlightNum', 'Description', 'AirlineID', 'UniqueCarrier']].head()

,FlightNum,Description,AirlineID,UniqueCarrier
2842,5621,ExpressJet Airlines Inc.,20366,EV
5081,3682,Envoy Air,20398,MQ
5082,3682,Envoy Air,20398,MQ
5083,3682,Envoy Air,20398,MQ
5084,3682,Envoy Air,20398,MQ


In [16]:
def map_airlines(x):
    if x.Description == 'Envoy Air':
        return 'American Airlines Inc.'
    
    elif x.Description == 'ExpressJet Airlines Inc.':
        if 2500 >= x.FlightNum <= 2574 :
            return 'American Airlines Inc.'
        if 3255 >= x.FlightNum <= 4868 :
            return 'United Air Lines Inc.'
        if 4869 >= x.FlightNum <= 5632 :
            return 'Delta Air Lines Inc.'
        if 5660 >= x.FlightNum <= 6189 :
            return 'United Air Lines Inc.'
    
    elif x.Description == 'SkyWest Airlines Inc.':
        if  2575 >= x.FlightNum <= 2649 : 
            return 'American Airlines Inc.'
        elif 2901 >= x.FlightNum <= 2974: 
            return 'US Airways Inc.'
        elif 3448 >= x.FlightNum <= 3449:  
            return 'Alaska Airlines Inc.'
        elif 4438 >= x.FlightNum <= 4859:
            return 'Delta Air Lines Inc.'
        elif 4965 >= x.FlightNum <= 65439:
            return 'United Air Lines Inc.'
        elif 6550 >= x.FlightNum <= 6629:
            return 'US Airways Inc.'
        elif 7362 >= x.FlightNum <= 7439:
            return 'Delta Air Lines Inc.'

    return x.Description

#Change the Description field according to the function map_airlines above.
data_toproutes['Description'] = data_toproutes.apply(map_airlines, axis=1)

In [17]:
#Look at the observations that were changed
subset_changed = data_toproutes[ (data_toproutes.UniqueCarrier == 'OO') | (data_toproutes.UniqueCarrier == 'MQ') | (data_toproutes.UniqueCarrier == 'EV')]
subset_changed[['FlightNum', 'Description', 'AirlineID', 'UniqueCarrier']]

#Look to see if all descriptions were changed for the regional carriers.
data_toproutes[(data_toproutes.Description == 'ExpressJet Airlines Inc.') 
                         | (data_toproutes.Description == 'Envoy Air' )
                         | (data_toproutes.Description == 'SkyWest Airlines Inc.' )].head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div5WheelsOff,Div5TailNum,Unnamed: 109,Code,Description,Des_abbrev,lat_origin,lon_origin,lat_dest,lon_dest


In [18]:
#Now look at what airlines are in the dataset:
print data_toproutes.Description.unique()

['Delta Air Lines Inc.' 'Southwest Airlines Co.'
 'AirTran Airways Corporation' 'United Air Lines Inc.'
 'American Airlines Inc.' 'Frontier Airlines Inc.' 'Spirit Air Lines'
 'JetBlue Airways' 'US Airways Inc.' 'Hawaiian Airlines Inc.'
 'Virgin America']


In [19]:
#Check how many flights are in the dataset.
len(data_toproutes)

200880

In [20]:
#Remove the flights that remain as being operated by regional air carriers, 
#since it is not possible to map these to the actual airlines that passengers booked their tickets with.
data_toproutes = data_toproutes[~data_toproutes['Description'].isin(['ExpressJet Airlines Inc.', 'SkyWest Airlines Inc.','Envoy Air'])]

In [21]:
#Check how many flights are in the dataset after removing.
len(data_toproutes)

200880

###4. Calculate Target Time 
#### Target time is based on the distance between the two airports, the east-west distance (to account for jet stream) and an added 43.2 minutes for taxi before take-off and after landing. 
#### Methodology adopted from FiveThirtyEight. For details see: http://fivethirtyeight.com/features/how-we-found-the-fastest-flights/#fn-2

####i. Calculate Target Time for each route

In [22]:
#Look at the different departure and arrival time variables to see how they differ.
data_toproutes[['CRSDepTime','DepTime', 'DepTimeBlk' ,'CRSArrTime','ArrTime', 'CRSElapsedTime','ActualElapsedTime']].head()

,CRSDepTime,DepTime,DepTimeBlk,CRSArrTime,ArrTime,CRSElapsedTime,ActualElapsedTime
0,1700,1713,1700-1759,1933,1947,153,154
1,1300,1254,1300-1359,1532,1516,152,142
2,1600,1558,1600-1659,1836,1825,156,147
3,600,557,0600-0659,825,820,145,143
4,1859,1859,1800-1859,2127,2117,148,138


In [23]:
#Calculate Target Time based on methodology from FiveThirtyEight
data_toproutes['target_time']=.117*data_toproutes.Distance + .517*(data_toproutes.lon_origin-data_toproutes.lon_dest) + 43.2

#### Sanity check the Target Time for flights SFO-LAX. According to FiveThirtyEight the Target Time should be 1 hour and 19 minutes.

In [24]:
data_toproutes[(data_toproutes['Origin']=='SFO') & (data_toproutes['Dest']=='LAX')].head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div5TailNum,Unnamed: 109,Code,Description,Des_abbrev,lat_origin,lon_origin,lat_dest,lon_dest,target_time
109910,2014,2,6,1,7,2014-06-01,AA,19805,AA,N3ENAA,...,NaN,NaN,19805,American Airlines Inc.,AA,37.618972,-122.374889,33.942536,-118.408075,80.578157
109911,2014,2,6,2,1,2014-06-02,AA,19805,AA,N3LLAA,...,NaN,NaN,19805,American Airlines Inc.,AA,37.618972,-122.374889,33.942536,-118.408075,80.578157
109912,2014,2,6,3,2,2014-06-03,AA,19805,AA,N3BGAA,...,NaN,NaN,19805,American Airlines Inc.,AA,37.618972,-122.374889,33.942536,-118.408075,80.578157
109913,2014,2,6,4,3,2014-06-04,AA,19805,AA,N3ASAA,...,NaN,NaN,19805,American Airlines Inc.,AA,37.618972,-122.374889,33.942536,-118.408075,80.578157
109914,2014,2,6,5,4,2014-06-05,AA,19805,AA,N3HBAA,...,NaN,NaN,19805,American Airlines Inc.,AA,37.618972,-122.374889,33.942536,-118.408075,80.578157


###5. Calculate Time Difference as the difference between the Actual time a flight took and the target time. 
#### I do not use the ActualElapsedTime variable because it is defined as time from leaving gate to time of arrival at the gate at the destination. This would not account for a flight that is delayed at the departure gate. To get the time from the scheduled departure to the time of the arrival at the gate at the destination I add _DepDelay_. Note that _DepDelay_ can be both negative and positive, meaning it will take into account for both times when the aircraft left the gate earlier than scheduled and later than scheduled. 
####This results in Travel time being measured from the originally scheduled departure time — the one listed on your ticket when you book the flight — until the plane pulls up at the gate at the destination airport

In [25]:
data_toproutes['Time_Diff']=(data_toproutes['ActualElapsedTime'] + data_toproutes['DepDelay']-data_toproutes['target_time'])
data_toproutes[['Time_Diff','ActualElapsedTime','DepDelay','target_time']].head()

,Time_Diff,ActualElapsedTime,DepDelay,target_time
0,29.188828,154,13,137.811172
1,-1.811172,142,-6,137.811172
2,7.188828,147,-2,137.811172
3,2.188828,143,-3,137.811172
4,0.188828,138,0,137.811172


#### Get summary stats for the Time Diff variable. 

In [26]:
data_toproutes.Time_Diff.describe()

count    197048.000000
mean         11.943984
std          41.239886
min         -53.848271
25%          -9.855439
50%           1.522171
75%          19.504738
max        1770.362113
Name: Time_Diff, dtype: float64

In [ ]:
data_forhist = data_toproutes[~data_toproutes['Time_Diff'].isnull()]
plt.hist(data_forhist['Time_Diff'], bins=100)
plt.title("Time Difference between Actual and Target Time")
plt.xlabel("Actual Time (relative to Target)")
plt.ylabel("Frequency")
plt.xlim([-100,300]) # set the xlim to xmin, xmax  
plt.show()


###6. Remove outliers by counting only flights that arrived at their destination without a diversion. 
####i. Remove cancelled and diverted flights. I am interested in seeing how baggage fees may affect 'regular delays', and not the more extreme cases.

In [ ]:
#Check how many flights are in the dataset.
len(data_toproutes)

In [ ]:
data_toproutes=data_toproutes[(data_toproutes['Cancelled']!=1)]
arrived_toproutes=data_toproutes[(data_toproutes['Diverted']!=1)]


In [ ]:
#Check how many flights are in the dataset after removing the cancelled and diverted flights.
len(data_toproutes)

In [ ]:
#List the column headers in the dataset to see what variables are associated with diversion 
#and can be deleted since they will no longer be applicable.
list(arrived_toproutes) 

In [ ]:
#Remove the columns that are irrelevant
arrived_toproutes.divcolumns = arrived_toproutes.loc[:,'DivAirportLandings':'Div5TailNum' ] #Create an object for the columns that can be removed
arrived_toproutes.drop(arrived_toproutes.divcolumns, axis=1, inplace=True) 


In [ ]:
list(arrived_toproutes) 

####ii. Do some data exploration to see how target time relates to the actual Travel Time

In [ ]:
#Do some exploration and look at the most delayed flights.
data_route_sorted= data_toproutes.sort('Time_Diff', axis=0 , ascending=False)
super_late = data_route_sorted.head(10)
super_late[['Year','Month','Carrier','DepDelay',
            "Origin", 'Dest',
            'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
            'ActualElapsedTime','ArrDelay','ArrDelayMinutes','ArrDel15','ArrivalDelayGroups',
            'Origin','Dest','target_time','Time_Diff']]

In [ ]:
data_delay_sorted= data_toproutes.sort('DepDelay', axis=0 , ascending=False)
super_delay = data_delay_sorted[(data_delay_sorted['DepDelay'] > 100)]
super_delay[['Year','Month','Carrier','DepDelay',
            'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
            'ActualElapsedTime','ArrDelay','ArrDelayMinutes','ArrDel15','ArrivalDelayGroups',
            'Origin','Dest','target_time','Time_Diff']].head(10)

In [ ]:
#Plot Target Times against Travel Times (the time from the scheduled departure to the time of arriving at 
#the gate at the destination). 
%matplotlib inline
data_toproutes['travel_time']=(data_toproutes['ActualElapsedTime'] + data_toproutes['DepDelay'])
g = sns.FacetGrid(data_toproutes, col="Description", hue="Month",col_wrap=3,ylim=(0, 2000))
g.map(plt.scatter, "target_time", "travel_time")
g.add_legend();


In [ ]:
airline_routes=data_toproutes[["Origin", "Dest", "Description","travel_time"]]
grouped =airline_routes.groupby(["Description","Origin", "Dest"])
airline_times = grouped["travel_time"].agg([np.size, np.mean, np.min, np.max]).reset_index()
airline_times.rename(columns = {'size': 'N_Flights', 'mean': 'AverageTravelTime', 'amin' : 'MinElapsedTime', 'amax': 'MaxElapsedTime'}, inplace = True)
airline_times.head()

##C. Build the Multiple Linear Regression model

###1. Prepare the features, which will be dummy variables for the origin airport, the destination airport, the airline and the existance of bag fees.

   ####a. Create dummy variables for the airports and the airline

In [ ]:
for column in ['Origin', 'Dest','Description']: #Create dummy variables for these 3 categorical variables
    dummies = pd.get_dummies(arrived_toproutes[column], prefix=column).iloc[:, 1:] #Remove one of the dummy variables to prevent collinearity
    arrived_toproutes[dummies.columns] = dummies
arrived_toproutes.head()


In [ ]:
arrived_toproutes.iloc[:,-25:].head()

In [ ]:
arrived_toproutes.head()


###2. Run a base case regression on the full data set. 

In [ ]:
#Separate the features and the outcome
Base_case_data = arrived_toproutes.loc[:,'Time_Diff':]
X = Base_case_data.iloc[:,1:-4]
y = arrived_toproutes['Time_Diff']

## fit a OLS model 
X = sm.add_constant(X)
basereg = sm.OLS(y, X).fit()


basereg.summary()

In [ ]:
y.mean()

###3. Add baggage fees for select airlines.
The data was collected manually from the following sources: 
    http://www.tripadvisor.com/AirlineFees 
    http://www.airfarewatchdog.com/blog/3801089/airline-baggage-fees-chart/
Note: 
1. Jetblue started charging a fee for the 1st checked bag starting in June. For purposes of this analysis, I assume that their previous fees structure was in place. This is reasonable since the data only contains flights in two months after the change (July and August 2015) and seats on these flights could have been booked prior to the fee change. 
2. Spirit Airlines does not have a very transparent fee structure and they start charging overweight charges already at 40lbs. For this analysis, I adopt the fees (at check-in) summarized here: http://www.seatguru.com/airlines/Spirit_Airlines/baggage.php

In [ ]:
Airline_Fees = pd.read_csv("data/AIRLINE_FEES_man.csv", skipinitialspace=True)
Airline_Fees.head()
arrived_toproutes= pd.merge(arrived_toproutes, Airline_Fees, how='left', on='Description')

#Make sure data on bag fees are not missing for any of the airlines in our data set.
arrived_toproutes[arrived_toproutes['Bag_1'].isnull()]

#List the unique values for Bag_1.  
arrived_toproutes.Bag_1.unique()



In [ ]:
#Remove the two other variables that came in the AIRLINE_FEES data set. 
arrived_toproutes.drop(['Code_y','Abbre'], axis=1, inplace=True)

####Create a dummy variable for the existence of a fee for checking the first bag. 

In [ ]:
arrived_toproutes['Bag1_Dummy']=arrived_toproutes['Bag_1'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:

arrived_toproutes.head()

In [ ]:
arrived_toproutes['Bag1_Dummy'].describe()

###4. Run a regression adding the existence of a Bag Fee(on the first checked bag) as an added feature.

In [ ]:
#Run a multiple linear regression model with the Bag dummy as an added feature for data across all months.
all_regmdata = arrived_toproutes.loc[:,'Time_Diff':] 
all_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

X = all_regdata.iloc[:,1:] #All data used for the regression will be found one column to the right of 'Time_Diff'
y = arrived_toproutes['Time_Diff']

X = sm.add_constant(X)
est = sm.OLS(y, X).fit() #Fit the model
    
est.summary

In [ ]:
arrived_toproutes['Time_Diff'].describe()

In [ ]:
#Run a multiple linear regression model with the Bag dummy as an added feature for each month individually.
monthvar = 1
coeff_df = pd.DataFrame()
tvalues_df = pd.DataFrame()
s_error_df = pd.DataFrame()
nobs_df = pd.DataFrame()
pvalues_df = pd.DataFrame()

while ( monthvar <= 12): 
    monthly_filter = arrived_toproutes [(arrived_toproutes['Month']==monthvar)]
    monthly_regdata = monthly_filter.loc[:,'Time_Diff':] 
    monthly_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

    X = monthly_regdata.iloc[:,1:] #All data used for the regression will be found one column to the right of 'Time_Diff'
    y = monthly_filter['Time_Diff']
    
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit() #Fit the model
    
    #Create a DataFrame with all the coefficients
    coeff_new = est.params
    coeff_df=coeff_df.append(coeff_new, ignore_index=True)
    
    #Create a DataFrame with all the T-Values
    tvalues_new = est.tvalues
    tvalues_df=tvalues_df.append(tvalues_new, ignore_index=True)
    
    #Create a DataFrame with all the Standard Errors
    s_error_new = est.bse
    s_error_df=s_error_df.append(s_error_new, ignore_index=True)
    
    #Create a DataFrame with all the P-values
    pvalues_new = est.pvalues
    pvalues_df=pvalues_df.append(pvalues_new, ignore_index=True)
    
    monthvar = monthvar + 1

In [ ]:
coeff_df



In [ ]:
pvalues_df

In [ ]:
#Combine the coefficients and pvalues for the bag1_dummy in one dataframe. 
Bag1_dum_results = pd.concat([coeff_df['Bag1_Dummy'], pvalues_df['Bag1_Dummy']], axis=1, ignore_index=False,)
Bag1_dum_results.columns = ['Coeff', 'PValue']
Bag1_dum_results

###5. Modify the dependent variable to be calculated without allowing early departures to subtract from the travel time. This is obtained by using the variable _DepDelayMinutes_ instead if _DepDelay_ (added to _ActualElapsedTime_)

In [ ]:
#Calculate the Time Difference according to a different methodology
arrived_toproutes['Time_Diff_noearly'] = (data_toproutes['ActualElapsedTime'] 
                                          + data_toproutes['DepDelayMinutes']
                                          - data_toproutes['target_time'])

In [ ]:
#Run a multiple linear regression model with the Bag dummy as an added feature, and with the new version of calculating the Time Difference
#for data across all months.
noearly_regdata = arrived_toproutes.loc[:,'Time_Diff':] 
noearly_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

X = noearly_regdata.iloc[:,1:-1] #All data used for the regression will be found one column to the right of 'Time_Diff'
y = arrived_toproutes['Time_Diff_noearly']
    

X = sm.add_constant(X)
est = sm.OLS(y, X).fit() #Fit the model
    
est.summary

In [ ]:
#Run the multiple linear regression model again with the Bag dummy as an added feature. 

monthvar = 1
coeff_df = pd.DataFrame()
tvalues_df = pd.DataFrame()
s_error_df = pd.DataFrame()
nobs_df = pd.DataFrame()
pvalues_df = pd.DataFrame()


while ( monthvar <= 12): 
    monthly_filter = arrived_toproutes [(arrived_toproutes['Month']==monthvar  )]
    monthly_filter = monthly_filter[pd.notnull(monthly_filter['Time_Diff_noearly'])]
  
    monthly_regdata = monthly_filter.loc[:,'Time_Diff':] 
    monthly_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

    X = monthly_regdata.iloc[:,1:-1] #All data used for the regression will be found one column to the right of 'Time_Diff'
    y = monthly_filter['Time_Diff_noearly']
    
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit() #Fit the model
    
    #Create a DataFrame with all the coefficients
    coeff_new = est.params
    coeff_df=coeff_df.append(coeff_new, ignore_index=True)
    
    #Create a DataFrame with all the T-Values
    tvalues_new = est.tvalues
    tvalues_df=tvalues_df.append(tvalues_new, ignore_index=True)
    
    #Create a DataFrame with all the Standard Errors
    s_error_new = est.bse
    s_error_df=s_error_df.append(s_error_new, ignore_index=True)
    
    #Create a DataFrame with all the P-values
    pvalues_new = est.pvalues
    pvalues_df=pvalues_df.append(pvalues_new, ignore_index=True)
    
    monthvar = monthvar + 1

In [ ]:
arrived_toproutes['Time_Diff_noearly'].describe()
#This new methodology increases the average Time Difference (as expected)


In [ ]:
coeff_df

In [ ]:
pvalues_df

In [ ]:
#Combine the coefficients and pvalues for the bag1_dummy in one dataframe. 
Bag1_noearly_results = pd.concat([coeff_df['Bag1_Dummy'], pvalues_df['Bag1_Dummy']], axis=1, ignore_index=False,)
Bag1_noearly_results.columns = ['Coeff', 'PValue']
Bag1_noearly_results 

###6. Modify the dependent variable to be _DepDelayMinutes_.

In [ ]:
#Run a multiple linear regression model with the Bag dummy as an added feature, and with the new version of calculating the Time Difference
#for data across all months.
Delay_regdata = arrived_toproutes.loc[:,'Time_Diff':] 
Delay_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

X = Delay_regdata.iloc[:,1:-1] #All data used for the regression will be found one column to the right of 'Time_Diff'
y = arrived_toproutes['DepDelayMinutes']
    

X = sm.add_constant(X)
est = sm.OLS(y, X).fit() #Fit the model
    
est.summary

In [ ]:
#Run the multiple linear regression model again with the Bag dummy as an added feature. 

monthvar = 1
coeff_df = pd.DataFrame()
tvalues_df = pd.DataFrame()
s_error_df = pd.DataFrame()
nobs_df = pd.DataFrame()
pvalues_df = pd.DataFrame()


while ( monthvar <= 12): 
    monthly_filter = arrived_toproutes [(arrived_toproutes['Month']==monthvar  )]
    monthly_filter = monthly_filter[pd.notnull(monthly_filter['DepDelayMinutes'])]
  
    monthly_regdata = monthly_filter.loc[:,'Time_Diff':] 
    monthly_regdata.drop(['Bag_1','Bag_2'], axis=1, inplace=True) #Remove the actual baggage fees, to only use the Bag1_dummy as a feature

    X = monthly_regdata.iloc[:,1:-1] #All data used for the regression will be found one column to the right of 'Time_Diff'
    y = monthly_filter['DepDelayMinutes']
    
    est = sm.OLS(y, X).fit() #Fit the model
    
    #Create a DataFrame with all the coefficients
    coeff_new = est.params
    coeff_df=coeff_df.append(coeff_new, ignore_index=True)
    
    #Create a DataFrame with all the T-Values
    tvalues_new = est.tvalues
    tvalues_df=tvalues_df.append(tvalues_new, ignore_index=True)
    
    #Create a DataFrame with all the Standard Errors
    s_error_new = est.bse
    s_error_df=s_error_df.append(s_error_new, ignore_index=True)
    
    #Create a DataFrame with all the P-values
    pvalues_new = est.pvalues
    pvalues_df=pvalues_df.append(pvalues_new, ignore_index=True)
       
    monthvar = monthvar + 1

In [ ]:
arrived_toproutes['DepDelayMinutes'].describe()


In [ ]:
coeff_df

In [ ]:
pvalues_df

In [ ]:
#Combine the coefficients and pvalues for the bag1_dummy in one dataframe. 
DepDelay_results = pd.concat([coeff_df['Bag1_Dummy'], pvalues_df['Bag1_Dummy']], axis=1, ignore_index=False,)
DepDelay_results.columns = ['Coeff', 'PValue']
DepDelay_results 